In [242]:
from src.core import *
from src import DSL
import inspect
import typing

import asyncio

from poke_env.player.random_player import RandomPlayer
from poke_env.player.utils import cross_evaluate
from poke_env.player_configuration import PlayerConfiguration
from poke_env.server_configuration import LocalhostServerConfiguration
from poke_env.utils import to_id_str
from tabulate import tabulate

import trueskill
import copy
from trueskill import Rating, rate_1vs1, quality_1vs1
from tqdm.notebook import tqdm
import itertools
import numpy as np
import pandas as pd
from pprint import pprint

In [246]:
class PlayerWrapper(Player):
    def __init__(self, *args, script=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.script = script
        self.rating = Rating()
        
    def choose_move(self, battle):
        try:
            move = self.script.choose_move(battle)
            return self.create_order(move)
        except Exception as e:
            print(e)
            return self.choose_random_move(battle)
            
    @classmethod
    def from_script(cls, script):
        script_name = script.__class__.__name__.replace('_', '')
#         script_name = get_node_id()
        return cls(
            script=script,
            player_configuration=PlayerConfiguration(script_name, None),
            battle_format="gen7randombattle",
            server_configuration=LocalhostServerConfiguration,
            max_concurrent_battles=10000,
        )

In [247]:
async def evaluate_population(population, num_games, verbose=True):
    players = population[:]
    players_lookup = {player.username: player for player in players}
    
    for _ in tqdm(range(num_games), disable=not verbose):
        random.shuffle(players)
        awaitables = []
        for p1, p2 in zip(players[0::2], players[1::2]):
            send = p1.send_challenges(
                opponent=to_id_str(p2.username),
                n_challenges=1,
                to_wait=p2.logged_in,
            )
            accept = p2.accept_challenges(
                opponent=to_id_str(p1.username),
                n_challenges=n_challenges
            )
            awaitables.append(send)
            awaitables.append(accept)
        await asyncio.gather(*awaitables)
        
    battles = sum((list(player.battles.values()) for player in players), [])
    
    # use TrueSkill
#     for battle in battles:
#         p1_dict, p2_dict = battle.players
#         p1 = players_lookup[p1_dict['username']]
#         p2 = players_lookup[p2_dict['username']]
        
#         if battle.won:
#             winner, loser = p1, p2
#         else:
#             winner, loser = p2, p1
#         winner.rating, loser.rating = rate_1vs1(winner.rating, loser.rating)
    

In [248]:
def get_elites(population, k):
    return sorted(population, key=lambda s: s.rating, reverse=True)[:k]

In [249]:
def mutate(tree, filter_=None):
    tree = copy.deepcopy(tree)
    candidates = anytree.search.findall(tree, filter_=filter_)
    if candidates:
        target = random.choice(candidates)
        target.children = []
        generate_tree(target)
    return tree

In [250]:
def get_random_population(k):
    return [PlayerWrapper.from_script(exec_tree(get_random_tree()))
            for _ in range(k)]

In [251]:
def get_population_stats(population):
    ratings = pd.Series([player.rating.mu for player in population])
    return ratings.describe()

In [252]:
num_elites = 3
population_cap = 64
epochs = 30
num_games = 3

population = get_random_population(k=population_cap)
generation_stats = []
for _ in tqdm(range(epochs)):
    await evaluate_population(population, num_games=num_games, verbose=False)
    generation_stats.append(get_population_stats(population))
    next_population = []
    next_population.extend(get_elites(population, num_elites))
    while len(next_population) < population_cap:
        parent = random.choice(population)
        child_tree = mutate(parent.script.tree)
        player = PlayerWrapper.from_script(exec_tree(child_tree))
        next_population.append(player)
    population = next_population
    for player in population:
        player.reset_battles()

In [260]:
print(sorted(population, key=lambda s: s.rating, reverse=True)[0].script.raw_script)


class Script_1673dbfd85594b64883845e3deec1824(Script):
    def choose_move(self, battle: Battle):
    
        available_moves = battle.available_moves
        if not available_moves:
            return random.choice(battle.available_switches)
            
        dsl = DSL(battle)
        move_scores = []
        for move in battle.available_moves:
            score = 0
            if ((dsl.player_base_speed() >= 201)):
                score += -1
            if ((dsl.check_move_sunny(move) and (dsl.move_base_power(move) <= 100))):
                score += 3
            move_scores.append(score)
            
        best_move = available_moves[move_scores.index(max(move_scores))]
        return best_move



In [253]:
for i, stat in enumerate(generation_stats):
    print(f"Generation #{i}")
    print(stat['mean'])
    print()
    print()

Generation #0
24.53175125285526


Generation #1
25.261722688486365


Generation #2
25.43930733808498


Generation #3
25.192599233364966


Generation #4
25.676552319956542


Generation #5
25.507149153603304


Generation #6
24.935915690480915


Generation #7
25.468579686021506


Generation #8
24.99644234485048


Generation #9
25.183236153844444


Generation #10
24.78367062579131


Generation #11
24.819767250713834


Generation #12
25.406090721908182


Generation #13
25.53504548564748


Generation #14
25.946255080263924


Generation #15
25.31194091999302


Generation #16
25.393260918553885


Generation #17
26.002730504801512


Generation #18
25.69446666956602


Generation #19
25.455299148586466


Generation #20
25.426940127292493


Generation #21
25.356914909965496


Generation #22
25.68322634919972


Generation #23
25.327248715394255


Generation #24
24.573262071249886


Generation #25
25.44144026299243


Generation #26
24.917163753506514


Generation #27
24.839076864680138


Generation 